In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import transformers
import torch
import pandas as pd

In [ ]:
# Set up the environment for efficient processing
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)


In [ ]:
# Model path and pipeline setup for LLaMA 3.2 3B Instruct
model_id = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cuda",
)

In [ ]:
# Prompt template to maximize disagreement
prompt_template = """
You are an argumentative writer who thrives on exploring controversial and conflicting ideas. Your task is to write a short essay on the topic: "{topic_name}".
The essay should:
- Present arguments that directly oppose each other.
- Explore the topic from polarized perspectives, emphasizing contradictions.
- Highlight flaws in all arguments, leaving room for interpretation.
- Be unique, bold, and thought-provoking.

Write the essay below:
"""

In [ ]:
# Load test data
test_file_path = "/kaggle/input/llms-you-cant-please-them-all/test.csv"  # Adjust this path as needed
test = pd.read_csv(test_file_path)

In [ ]:
# Process topics and generate essays in batches
batch_size = 3  # Adjust based on available GPU memory
predictions = []

In [ ]:
for i in range(0, len(test), batch_size):
    # Prepare batch data
    batch_topics = test.loc[i:i + batch_size - 1, "topic"].tolist()
    batch_prompts = [prompt_template.format(topic_name=topic) for topic in batch_topics]

    # Generate predictions for the batch
    outputs = pipeline(
        batch_prompts,  # Input prompts directly
        max_new_tokens=1024,  # Longer generation to allow more depth
        temperature=1.7,  # Higher temperature to encourage diverse outputs
        top_p=0.9,  # Nucleus sampling for creative diversity
        top_k=50,  # Limits to the top 50 token probabilities
    )

In [ ]:
batch_essays = [output["generated_text"] if isinstance(output, dict) else output for output in outputs]
predictions.append(batch_essays)

In [ ]:

# if i < 3 * batch_size:
#         for topic, essay in zip(batch_topics, batch_essays):
#             print("Topic: ", topic)
            
#             print("Generated Essay: ", essay)
#             print("\n\n***********************\n\n")
#             print(type(essay))
new_predictions=[]
if i < 3 * batch_size:
    for topic, essay in zip(batch_topics, batch_essays):
        print("Topic: ", topic)
        # Extract the essay text from the dictionary
        if isinstance(essay, list) and len(essay) > 0 and 'generated_text' in essay[0]:
            essay_text = essay[0]['generated_text']
        else:
            essay_text = "Invalid format for essay"
        
        print("Generated Essay: ", essay_text)
        print("\n\n***********************\n\n")
        print(type(essay_text))
        new_predictions.append(essay_text)


In [ ]:
print(type(new_predictions))

In [ ]:
submission = pd.read_csv('/kaggle/input/llms-you-cant-please-them-all/sample_submission.csv')
submission.head()

In [ ]:
# if len(predictions) > len(submission):
#     predictions = predictions[:len(submission)]
submission['essay'] = new_predictions
submission.head()

In [ ]:
submission.to_csv('submission.csv',index = False)
print("submission file saved succesfully!")

In [ ]:
submission["essay"][0]